In [1]:
import arcpy
import pandas as pd
import arcgis
import numpy as np

In [3]:
Output_GDB = r"N:\projects\2022\225180C215139 NTUA Tsaile Wheatfields Chapter Fiber Optic Line\Biology\Graphics-GIS\GIS\Map_Data.gdb"

In [4]:
output_folder = r"N:\projects\2022\225180C215139 NTUA Tsaile Wheatfields Chapter Fiber Optic Line\Biology\Graphics-GIS\GIS\Analysis"

In [5]:
arcpy.env.workspace = "CURRENT"

In [6]:
arcpy.env.overwriteOutput = True

In [7]:
run_data_setup = True

In [8]:
def data_setup():
    ''' Clips raster to project area.
        Converts raster to polygon
        Clips polygons to project area
    '''
    arcpy.management.Clip('az_landcover', 'Project_Area', rf"{Output_GDB}\Landcover_Clipped")
    arcpy.conversion.RasterToPolygon('Landcover_Clipped', rf"{Output_GDB}\Landcover_Polygons", raster_field="DESCRIPTION")
    arcpy.analysis.Clip('Landcover_Polygons', 'Project_Area', rf"{Output_GDB}\Polygons_Clipped")

In [9]:
if run_data_setup:
    data_setup()

In [10]:
# Import Clipped Polygons as spatially enabled dataframe 

sdf = pd.DataFrame.spatial.from_featureclass(rf"{Output_GDB}\Polygons_Clipped")

In [11]:
# Convert to geo series to get area and add to sdf

gsa = arcgis.features.GeoSeriesAccessor(sdf['SHAPE'])
sdf['Polygon_Area'] = gsa.area

In [12]:
# Aggregate based on Description and Polygon area sum

sdf = sdf.groupby(by='DESCRIPTION', as_index=False).agg({'Polygon_Area': 'sum'})
sdf.head()

,DESCRIPTION,Polygon_Area
0,Colorado Plateau Pinyon-Juniper Woodland,54966.873350
1,Inter-Mountain Basins Big Sagebrush Shrubland,29328.449003
2,Inter-Mountain Basins Semi-Desert Shrub Steppe,1004.440265
3,Invasive Perennial Grassland,60.860957
4,Rocky Mountain Ponderosa Pine Woodland,37004.975329


In [13]:
# Convert to Acres and add to df

sdf['MRS_Acres'] = sdf['Polygon_Area'] *  0.00024711 
sdf.head()

,DESCRIPTION,Polygon_Area,MRS_Acres
0,Colorado Plateau Pinyon-Juniper Woodland,54966.873350,13.582864
1,Inter-Mountain Basins Big Sagebrush Shrubland,29328.449003,7.247353
2,Inter-Mountain Basins Semi-Desert Shrub Steppe,1004.440265,0.248207
3,Invasive Perennial Grassland,60.860957,0.015039
4,Rocky Mountain Ponderosa Pine Woodland,37004.975329,9.144299


In [14]:
# Add field as column of whole for percentage

sdf['Percent_of_total'] = (sdf['MRS_Acres'] / sdf['MRS_Acres'].sum()) * 100
display(sdf)

,DESCRIPTION,Polygon_Area,MRS_Acres,Percent_of_total
0,Colorado Plateau Pinyon-Juniper Woodland,54966.873350,13.582864,44.920201
1,Inter-Mountain Basins Big Sagebrush Shrubland,29328.449003,7.247353,23.967887
2,Inter-Mountain Basins Semi-Desert Shrub Steppe,1004.440265,0.248207,0.820852
3,Invasive Perennial Grassland,60.860957,0.015039,0.049737
4,Rocky Mountain Ponderosa Pine Woodland,37004.975329,9.144299,30.241322


In [173]:
# Append total to the bottom of df

sdf.loc['Grand_Total'] = sdf[['MRS_Acres', 'Polygon_Area','Percent_of_total']].sum()
display(sdf)

,DESCRIPTION,Polygon_Area,MRS_Acres,Percent_of_total
0,Inter-Mountain Basins Big Sagebrush Shrubland,1.502241e+04,3.712188,0.600631
1,Inter-Mountain Basins Montane Sagebrush Steppe,7.584645e+05,187.424160,30.325176
2,Inter-Mountain Basins Subalpine Limber-Bristle...,9.760317e+04,24.118720,3.902402
3,Mediterranean California Alpine Bedrock and Scree,5.281196e+04,13.050363,2.111545
4,Rocky Mountain Subalpine Dry-Mesic Spruce-Fir ...,2.603034e+05,64.323562,10.407534
5,Rocky Mountain Subalpine Mesic Spruce-Fir Fore...,6.093099e+04,15.056658,2.436163
6,Sierra Nevada Cliff and Canyon,5.416110e+03,1.338375,0.216549
Grand_Total,NaN,2.501105e+06,618.048054,100.000000


In [15]:
sdf.to_excel(rf"{output_folder}\Total_Acres.xlsx")